- title: Занятие 8
- author: Dmitrii Kononov
- date: 2023-03-23
- test_link: http://olymp3.vdi.mipt.ru/cgi-bin/new-client?contest_id=10205
- test_comment: Задание №5    
- slug: s2_lab08
- ipynb_url: download/s2_lab08.ipynb

# Занятие 8: Хэш-функции и их применение.

## Цели работы
1. Определение Хэш-функции.
2. Проверка целостности файлов.
3. Хранение паролей.
4. Поиск подстроки в строке алгоритмом Рабина—Карпа.

## Хэш-функция
Хэш-функция — эта функция, которая преобразует строку (последовательность символов) в число. Основным требованием к хэш-функции является требование равномерного распределения значений. Отсюда и название — `хэширование` (`hashing` от англ. «перемешивание»).

Вторым требованием к Хэш-функции является уникальность значений функции для всех значений аргумента. На практике возможно соответствие значения функции нескольким значениям аргумента — так называемая **коллизия**.

Важным свойством с точки зрения приложений кибербезопасности Хэш-функции является невозможность восстановления значения аргумента по известному значению функции за разумное время.
### Библиотека hashlib
Для вычисления хэш-функции в python используются функции библиотеки [hashlib](https://docs.python.org/3/library/hashlib.html). Например, код ниже вычисляет хэш-функцию строки `Hello world!` с помощью алгоритма *SHA*. Запустите код в ячейке ниже:

In [1]:
import hashlib
m = hashlib.sha256()
m.update(b"Hello world!")
print(m.hexdigest())

c0535e4be2b79ffd93291305436bf889314e4a3faec05ecffcbb7df31ad9e51a


## Применение Хэш-функции
### Проверка целостности файлов
Одно из распространенных применений Хэш-функции — это подтверждение целостности файлов при передаче их по каналам связи/скачивания с веб-ресурсов. Если значение Хэш-функции полученного по сети файла совпадает с эталонным значением, значит файл передан без искажений.
Код в ячейке ниже осуществляет чтение файла и вычисление MD5 значения Хэш-функции:
```python
import hashlib
md5_hash = hashlib.md5()
with open("file.txt", "rb") as f:
    while True:
        data = f.read(2048)
        if not data:
            break
        md5_hash.update(data)
print(md5_hash.hexdigest())
```
**Задание 1 —** создайте текстовый файл и вычислите его md5 хэш с помощью примера кода выше.

In [3]:
import hashlib
md5_hash = hashlib.md5()
with open("file.txt", "rb") as f:
    while True:
        data = f.read(2048)
        if not data:
            break
        md5_hash.update(data)
print(md5_hash.hexdigest())

d41d8cd98f00b204e9800998ecf8427e


### Хранение паролей
Другим распространенным применением Хэш-функции является безопасное хранение паролей. В этом случае хранят не сами пароли, которые пользователь ввел, например, при регистрации на сайте, а результат вычисления их хэш-функции.

Затем на этапе аутентификации пользователя сравниваются хэш значения введенного пароля и хэш значение, сохраненное в базе.

Для генерации хэша пароля можно использовать функцию `pbkdf2_hmac()` из библиотеки `hashlib`. Функция `pbkdf2_hmac()` принимает пять параметров:
 - hash_name: алгоритм хеш дайджеста для HMAC;
 - password: пароль, превращенный в ключ;
 - salt: случайно сгенерированная соль;
 - iterations: итерации в вычислении (чем больше, тем длиннее вычисления);
 - dklen: длина ключа вывода (не обязательно).

Перед генерацией ключа с использованием `pbkdf2_hmac` нужно сгенерировать случайную соль. Смешивание исходного пароля с солью увеличивает число возможных вариантов перебора и затрудняет подбор пароля по известному хэш методом грубой силы. Использование соли требует чуть больше работы и хранении дополнительной последовательности байтов. Для генерации соли используется функция `os.urandom()`, которая возвращает случайные байты, используемые для шифрования. Соль складывается с паролем, чтобы ввод покрывал больший диапазон.

Пример кода генерации соли:
```python
import os
salt = os.urandom(32)
```
Параметр 32 является размером, возвращаемым в байтах.

Ниже пример кода генерации хэш-функции от пароля:
```python
import hashlib
import os
 
salt = os.urandom(32)
password = 'QWasZX12'
hash_pass = hashlib.pbkdf2_hmac('sha256', password.encode('utf-8'), salt, 100000) 
```
Использованы следующие параметры функции `hashlib.pbkdf2_hmac`:
 - 'sha256' — алгоритм хеширования
 - password.encode('utf-8') — пароль в виде байтов
 - salt — Соль
 - число итераций алгоритма хэширования, затрудняющее подбор пароля
 
Для последующего сравнения введенного пользователем пароля необходимо хранить в базе данных хэш и соль.

**Задание 2 —** Реализуйте пользовательский ввод пароля и сравнение с сохраненным эталоном.

In [4]:
import os
salt = os.urandom(32)

import hashlib
import os
 
salt = os.urandom(32)
password = input("Write pass: ")
hash_pass = hashlib.pbkdf2_hmac('sha256', password.encode('utf-8'), salt, 100000)

s = ""
while hash_pass != hashlib.pbkdf2_hmac('sha256', s.encode('utf-8'), salt, 100000):
    s = input()

### Поиск подстроки в строке алгоритмом Рабина—Карпа
В осеннем семестре рассматривался "наивный" алгоритм поиска подстроки в строке с квадратичным временем исполнения в наихудшем случае $O(n^{2})$, который заключался в последовательном наложении подстроки `p` на строку `s`.

На этом занятии будет рассмотрен алгоритм поиска подстроки `p` в строке `s` с линейным временем выполнения $O(n)$ — алгоритм Рабина—Карпа, основанный на применении Хэш-функции.

Алгоритм основан на том факте, что если строка-образец `p` (длины *m*) и подстрока длины *m* символов, начинающаяся с позиции *i* строки `s` совпадают, то должны совпадать и их хэш-функции. Если же строки разные, то и их хэш-функции почти наверняка отличаются. Одинаковые хэш-функции для разных строк должны встречаться настолько редко, что можно допустить время $O(m)$ на явную проверку идентичности строк при совпадении хэш-функций. Это сводит сложность поиска подстроки в строке к вычислениям $n-m+2$ значений хэш-функции плюс небольшое число сравнений сложности $O(m)$ строк с одинаковыми значениями хэш-функций.

Надлежащий выбор хэш-функции позволит тратить на ее вычисление для разных отрезков строки `s` время, меньшее $O(m)$. Таким правильным выбором является хеш-функция `m` символов строки `s`, начинающихся с символа `j`:
$H(s, j) = (\sum_{i=0}^{m-1} \alpha^{m-(i+1)}\times ord(s_{i+j})) mod{q},$ где $ord(c)$ — ANSI-код символа `c`, q — простое число, а $\alpha$ — число от 0 до q-1.

Удобство выбора именно такой хэш-функции заключается в том, что $H(s, j+1)$ (т.е. хэш-функция от подстроки `s` длины `m`, сдвинутой на 1 символ) равна: $$H(s, j+1) = (\alpha(H(s, j)-\alpha^{m-1}ord(s_j)) + ord(s_{j+m})) mod {q}.$$
Другими словами, если нам уже известна хэш-функция $H(s, j)$ для подстроки, начинающейся в позиции `j`, то хэш-функцию подстроки с позиции `j+1` можно вычислить выполнив две операции умножения, одну операцию сложения, одну операцию вычитания и одну операцию вычисления остатка от деления.  
Одним из возможных эффективных методов выбора пары $\alpha$ и q [следующий](https://ru.wikipedia.org/wiki/Алгоритм_Рабина_—_Карпа): $\alpha = 2,$ q — случайное число из диапазона $[2...n^3]$.

Возможная реализация алгоритма Рабина-Карпа на *Python* выглядит следующим образом (принимает на вход строку `s` и образец `p`, возвращает индекс начала искомого образца в строке, либо -1, если образец не найден):
```python
def rabin_karp(s, p):
    n = len(s)
    m = len(p)
    hash_p = hash_func(p);
    for i in range(n-m+1):
        hash_s = hash_func(s[i:i+m])
        if hash_s == hash_p:
            if s[i:i+m] == p:
                return i
    return -1
```
**Задание 3 —** Реализуйте функцию `hash_func()` в соответствии с описанием выше и скорректируйте код функции `rabin_karp(s, p)` из примера выше, чтобы получить работающую реализацию алгоритма Рабина-Карпа. Продемонстрируйте ее работу.

In [81]:
import random


buf_len = 10

def binpow(a, n):
	global buf_len
	res = None

	if not hasattr(binpow, "buf"):
		binpow.buf = [0] * buf_len
		binpow.buf_ans = [0] * buf_len
		binpow.count = 0
	
	if (a, n) in binpow.buf:
		res = binpow.buf_ans[binpow.buf.index((a, n))]
	elif (a, n + 1) in binpow.buf:
		res = binpow.buf_ans[binpow.buf.index((a, n + 1))] / a
	elif (a, n - 1) in binpow.buf:
		res = binpow.buf_ans[binpow.buf.index((a, n - 1))] * a
	else:
		res = 1
		while n:
			if n & 1:
				res *= a
			a *= a
			n >>= 1
	
	binpow.buf[binpow.count % buf_len] = (a, n)
	binpow.buf_ans[binpow.count % buf_len] = res
	binpow.count += 1
	
	return res


In [79]:
a = 2
q = random.choice([2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397, 401, 409, 419, 421, 431, 433, 439, 443, 449, 457, 461, 463, 467, 479, 487, 491, 499, 503, 509, 521, 523, 541, 547, 557, 563, 569, 571, 577, 587, 593, 599, 601, 607, 613, 617, 619, 631, 641, 643, 647, 653, 659, 661, 673, 677, 683, 691, 701, 709, 719, 727, 733, 739, 743, 751, 757, 761, 769, 773, 787, 797, 809, 811, 821, 823, 827, 829, 839, 853, 857, 859, 863, 877, 881, 883, 887, 907, 911, 919, 929, 937, 941, 947, 953, 967, 971, 977, 983, 991, 997])

num = lambda x: ord(x) - ord('A')

def hash_func(s):
    global a, q
    m = len(s)
    h = 0
    for i in range(m):
        h += binpow(a, m - (i + 1)) * num(s[i])
    
    return h

def hash_next(m, last_h, for_del, for_add):
    global a, q
    return (a * (last_h - binpow(a, m - 1) * num(for_del)) + num(for_add))


In [80]:
def rabin_karp(s, p):
    n = len(s)
    m = len(p)
    hash_p = hash_func(p)
    for i in range(n-m+1):
        if i == 0:
            hash_s = hash_func(s[:m])
        else:
            hash_s = hash_next(len(p), hash_s, s[i - 1], s[i + m - 1])
        
        if hash_s == hash_p:
            if s[i:i+m] == p:
                return i
    return -1

rabin_karp("ABCD", "CD")

2